<br>

# Introdução

In [ ]:
import os
import shutil
import folium
import fnmatch
import pandas as pd
import geopandas as gpd

from osgeo import ogr
from folium import plugins
from zipfile import ZipFile

In [ ]:
#!pip install traquitanas --upgrade
from traquitanas.geo import layers as tt

In [ ]:
maps_path = os.path.join('..', 'maps')
data_path = os.path.join('..', 'data')
input_path = os.path.join(data_path, 'input')
output_path = os.path.join(data_path, 'output')
temp_path = os.path.join(data_path, 'temp')

shutil.rmtree(temp_path, ignore_errors=True)
shutil.rmtree(output_path, ignore_errors=True)

os.makedirs(output_path, exist_ok=True)
os.makedirs(temp_path, exist_ok=True)
os.makedirs(maps_path, exist_ok=True)

<br>

Inicialmente extraímos todos os arquivos de dentro do zip "Shape.zip".

In [ ]:
zipfile_path = os.path.join(input_path, 'Shapes.zip')

In [ ]:
with ZipFile(zipfile_path, 'r') as zip_obj:    
    zip_obj.extractall(temp_path)

<br>

Uma vez com os arquivos soltos, foi possível observar grande desorganização dos arquivos. Não há qualquer padrão de nomenclatura. Alguns *shapefiles* existem para algumas nascentes, enquanto para outra snão existe.<br>
Como o objetivo principal do trabalho é identificar a localização das 56 nascentes que são mencionadas no TAC, optou-se por concentrar os esforços nestas feições.

In [ ]:
list_shps = []

for root, dirs, files in os.walk(temp_path):
    if os.path.basename(root).startswith('Nascente'):
        print('')
        print(root)
        for file in fnmatch.filter(files, '*.shp'):
            print('> ' + file)
            list_shps.append(file)

In [ ]:
list_shps = list(set(list_shps))
list_shps.sort()
list_shps

<br>

## Junta shapefiles por Nome

Com a lista dos possíveis nomes de shapefiles, juntou-se todos os arquivos.

In [ ]:
def merge_shp_by_name(shp, path):
    """
    path: pasta root que tem os shapefiles
    
    """
    # Add Fields
    print('\n{}'.format(shp))
    for root, dirs, files in os.walk(path):
        if os.path.basename(root).startswith('Nascente'):
            for file in fnmatch.filter(files, '*.shp'):
                if file in [shp]:
                    # Read
                    temp = os.path.join(root, file)
                    gdf = gpd.read_file(temp)
                    gdf.columns = [x.lower() for x in gdf.columns]

                    # Delete Fields
                    #for col in ['Id', 'path', 'nascente']:
                    gdf.drop([
                        'Id', 'path',
                        'nascente', 'Nascente',
                        'numero', 'x', 'y',
                        'BUFF_DIST', 'buff_dist',
                        'nascente_1'
                        'nascente_1', 'nascente_2',
                        'nascente_3', 'nascente_4',
                        'nascente_5'],
                        axis=1,
                        errors='ignore',
                        inplace=True,
                    )
                    
                    # Add Fields
                    gdf['path'] = temp
                    dirname = os.path.dirname(temp)
                    dirname = os.path.basename(dirname)
                    nascente_number = int(dirname.split(' ', maxsplit=1)[-1])
                    gdf['nascente'] = nascente_number

                    # Save Output
                    gdf.to_file(temp)

    # Lista Files
    print(gdf.columns)
    list_files = []                
    for root, dirs, files in os.walk(path):
        if os.path.basename(root).startswith('Nascente'):
            for file in fnmatch.filter(files, '*.shp'):
                if file in [shp]:
                    temp = os.path.join(root, file)
                    list_files.append(temp)

    # Combine all shapefiles
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )

    # Save Output
    gdf.to_file(os.path.join(output_path, shp))
    display(gdf)
    return gdf

In [ ]:
for shp in list_shps:
    gdf = merge_shp_by_name(shp, temp_path)

<br>

## Junta shapefiles Específicos

Após juntar todos os shapefiles em pastas, notei a existência de shapefiles que tem o mesmo tema, porém continham nomes distintos (por exemplo, "app.shp" e "App.shp").<br>
Esse problema na nomenclatura dos arquivos, impediu que os arquivos fossem unificados e, portanto, foi necessário escrever uma função que juntava os shapefiles, mantendo apenas o primeiro e deletando os restantes.

In [ ]:
def merge_shp_specific(list_files):
    """
    Une shapefiles específicos, sobrepondo o primeiro da lista...
    Deletando os demais...
    """
    # Concatena
    gdf = gpd.GeoDataFrame(
        pd.concat(
            [gpd.read_file(i) for i in list_files],
            ignore_index=True
        ),
        crs=gpd.read_file(list_files[0]).crs
    )

    # Deleta Others Files
    driver = ogr.GetDriverByName('ESRI Shapefile')
    for file in list_files[1:]:
        if os.path.exists(file):
            driver.DeleteDataSource(file)
            print('"{}" deletado!!'.format(file))
        else:
            print('"{}" não existe!'.format(file))

    # Save Output
    gdf.to_file(os.path.join(list_files[0]))

In [ ]:
merge_shp_specific(
    [
        os.path.join(output_path, 'APP.shp'),
        os.path.join(output_path, 'APP_nascente.shp'),
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(output_path, 'Area_conservacao.shp'),
        os.path.join(output_path, 'area_Conservacao.shp'),
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(output_path, 'area_drenagem.shp'),
        os.path.join(output_path, 'Area_Drenagem.shp'),
        os.path.join(output_path, 'Area_drenagem.shp'),        
    ]
)

In [ ]:
merge_shp_specific(
    [
        os.path.join(output_path, 'Recuperacao_curso_agua.shp'),
        os.path.join(output_path, 'Recuperacao_curso_dagua.shp'),
        os.path.join(output_path, 'Recuperacao_cursodagua.shp'),        
    ]
)

<br>

## Deleta Shapefiles

Após juntar o material que considerei útil para a etapa do meu trabalho, foi possível deletar o restante do material.

In [ ]:
def delete_shp(list_files):
    """
    Delete Lista de Shapefiles
    """
    # Deleta Others Files
    driver = ogr.GetDriverByName('ESRI Shapefile')
    for file in list_files[1:]:
        if os.path.exists(file):
            driver.DeleteDataSource(file)
            print('"{}" deletado!!'.format(file))
        else:
            print('"{}" não existe!'.format(file))


In [ ]:
list_files = [
    # Lixo
    os.path.join(output_path, 'APP_total.shp'),

    # Interessa?
    os.path.join(output_path, 'Geologia.shp'),
    #os.path.join(output_path, 'Nascente.shp'),
    os.path.join(output_path, 'Pontos_geofisica.shp'),
    os.path.join(output_path, 'Recomposicao.shp'),
    os.path.join(output_path, 'Recuperacao_nascente.shp'),
    os.path.join(output_path, 'Uso_solo_APP.shp'),
    os.path.join(output_path, 'Uso_solo_app_curso_dagua.shp'),
    os.path.join(output_path, 'Uso_solo_app_nasc.shp'),
    os.path.join(output_path, 'Uso_solo_area_drenagem.shp'),
    os.path.join(output_path, 'Uso_solo_curso_agua.shp'),
    os.path.join(output_path, 'Uso_solo_hidrografia.shp'),
]

delete_shp(list_files)

In [ ]:
shutil.rmtree(temp_path, ignore_errors=True)

<br>

# Maps

<br>

## Layer: Nascentes

In [ ]:
def add_lyr_nascentes(output_path):
    # Input
    gdf = gpd.read_file(os.path.join(output_path, 'Nascente.shp'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Layers
    lyr = folium.GeoJson(
        gdf,
        name='Nascentes',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['nascente'],
            aliases=['Nascente'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
    #     popup=folium.GeoJsonPopup(
    #         ['popup'],
    #         parse_html=False,
    #         max_width='400',
    #         show=False,
    #         labels=False,
    #         sticky=True,            
    #     ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

<br>

## Layer: Área Contribuição

In [ ]:
def add_lyr_areadrenagem(output_path):
    # Input
    gdf = gpd.read_file(os.path.join(output_path, 'area_drenagem.shp'))
    gdf = gdf.to_crs(epsg=4326)
    
    # Layers
    lyr = folium.GeoJson(
        gdf,
        name='Nascentes',
        smooth_factor=1.0,
        style_function=lambda x: {
            'fillColor': '#DC143C',
            'color': '#DC143C',
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['nascente'],
            aliases=['Nascente'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
    #     popup=folium.GeoJsonPopup(
    #         ['popup'],
    #         parse_html=False,
    #         max_width='400',
    #         show=False,
    #         labels=False,
    #         sticky=True,            
    #     ),
        embed=False,
        zoom_on_click=False,
        control=True,
        show=True,
    )
    return lyr

<br>

## Maps

In [ ]:
def get_map(input_shp):
    # Input
    gdf = gpd.read_file(input_shp)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 10
    max_zoom = 18
    
    # Create Map
    m = folium.Map(
        #zoom_start=10,
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        #zoom_delta=0.1,s
        min_lat=bounds[0][0]*(101/100),
        min_lon=bounds[0][1]*(101/100),
        max_lat=bounds[1][0]*(99/100),
        max_lon=bounds[1][1]*(99/100),
        tiles=None,
    )
    
    # Add Layers
    m.add_child(tt.add_lyr_google_terrain(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_hybrid(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_streets(min_zoom, max_zoom))
    m.add_child(tt.add_lyr_google_satellite(min_zoom, max_zoom))

    # Outros Layers
    m.add_child(add_lyr_nascentes(output_path))
    m.add_child(add_lyr_areadrenagem(output_path))
    #m.add_child(add_lyr_car())
    #m.add_child(add_lyr_divisa_municipal())
    #m.add_child(add_lyr_macrozona())
    #m.add_child(add_lyr_wms())
    
    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False
    ).add_to(m)
    return m

In [ ]:
# Create Map
m = get_map(os.path.join(output_path, 'Nascente.shp'))

# Results
m.save(os.path.join(maps_path, 'acp-399-01_map.html'))
m